In [1]:
import geopandas as gpd
import pandas as pd
from h3 import h3
from sqlalchemy import create_engine
from shapely.geometry import Polygon, Point
import numpy as np

In [2]:

def h3_from_row(row,res,x,y):
    '''
    Esta funcion toma una fila, un nivel de resolucion de h3
    y los nombres que contienen las coordenadas xy
    y devuelve un id de indice h3
    '''
    return h3.geo_to_h3(row[y],row[x],res = res)

def h3_indexing(df,res_list,lat='LATITUDE',lon='LONGITUDE'):
    """
    Esta funcion toma una tabla con dos pares de coordenadas para origen y destino
    Un nivel de resolucion h3
    y devuelve la tabla con los ids de h3
    """
    
    if len(res_list) == 1:
        res_list.append(res_list[0])
        
    for res in range(res_list[0],res_list[1] + 1):
        df['h3_res_' + str(res)] = df.apply(h3_from_row, axis=1, args=[res,lon,lat])

    return df

def vertices_cada_Xmetros(geom,metros):
    n_puntos = int((geom.length/metros)+1)
    percentiles = np.linspace(0,geom.length,n_puntos)
    return [geom.interpolate(percentil,normalized=False) for percentil in percentiles]

def convertir_recorridos_buses_paradas(fila,metros=400):
    paradas = vertices_cada_Xmetros(fila.geometry,metros = metros)
    paradas = gpd.GeoSeries(paradas).map(lambda g: Point(g.coords[0][0:2]))
    crs = 'EPSG:3857'
    gdf = gpd.GeoDataFrame(np.repeat(fila.LINEA,len(paradas)),crs=crs,geometry=paradas)
    gdf.columns=['LINEA','geometry']
    return gdf

In [27]:
subte = gpd.read_file('../carto/insumos/subterraneo-estaciones')
subte = subte.reindex(columns = ['LINEA','geometry'])
subte['MEDIO'] = 'SUB'

ffcc = gpd.read_file('../carto/insumos/rmba-ferrocarril-estaciones/')
ffcc = ffcc.loc[ffcc.Tipo=='Estación',['Línea','geometry']]
ffcc.columns = ['LINEA','geometry']
ffcc['MEDIO'] = 'TRE'

ffcc.head()

,LINEA,geometry,MEDIO
0,Mitre,POINT (-58.53165 -34.57368),TRE
1,Mitre,POINT (-58.54055 -34.56368),TRE
2,Mitre,POINT (-58.55608 -34.55091),TRE
5,Mitre,POINT (-58.57567 -34.53500),TRE
7,Mitre,POINT (-58.48792 -34.57472),TRE


In [36]:
bus_nac = gpd.read_file('../carto/insumos/lineas-nacionales/')
bus_nac = bus_nac.reindex(columns = ['LINEA','geometry'])
bus_nac = bus_nac.to_crs('EPSG:3857')

paradas_bus_nac = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_nac.iterrows()])
paradas_bus_nac.crs = 'EPSG:3857'
paradas_bus_nac = paradas_bus_nac.to_crs('EPSG:4326')
paradas_bus_nac['MEDIO'] = 'COL'
paradas_bus_nac.head()

,LINEA,geometry,MEDIO
0,1,POINT (-58.52931 -34.63934),COL
1,1,POINT (-58.52927 -34.63947),COL
2,1,POINT (-58.56432 -34.64013),COL
3,1,POINT (-58.56612 -34.63818),COL
4,1,POINT (-58.56971 -34.63831),COL


In [38]:
bus_prov = gpd.read_file('../carto/insumos/lineas-provinciales/')
bus_prov = bus_prov.reindex(columns = ['LINEA','geometry'])
bus_prov = bus_prov.to_crs('EPSG:3857')

paradas_bus_prov = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_prov.iterrows()])
paradas_bus_prov.crs = 'EPSG:3857'
paradas_bus_prov = paradas_bus_prov.to_crs('EPSG:4326')
paradas_bus_prov['MEDIO'] = 'COL'
paradas_bus_prov.head()
paradas_bus_prov

,LINEA,geometry,MEDIO
0,253,POINT (-58.52641 -34.63914),COL
1,253,POINT (-58.52996 -34.63944),COL
2,253,POINT (-58.53355 -34.63966),COL
3,253,POINT (-58.53714 -34.63994),COL
4,253,POINT (-58.54073 -34.64016),COL
...,...,...,...
110,322,POINT (-58.74170 -34.66883),COL
111,322,POINT (-58.73916 -34.66674),COL
112,322,POINT (-58.73581 -34.66564),COL
113,322,POINT (-58.73244 -34.66460),COL


In [42]:
pd.Series(paradas_bus_nac.LINEA.unique()).isin(paradas_bus_prov.LINEA.unique()).sum()

0

In [43]:
pd.Series(paradas_bus_prov.LINEA.unique()).isin(paradas_bus_nac.LINEA.unique()).sum()

0

In [46]:
bus_muni = gpd.read_file('../carto/insumos/lineas-municipales/')
bus_muni = bus_muni.reindex(columns = ['LINEA','geometry'])
bus_muni = bus_muni.to_crs('EPSG:3857')

paradas_bus_muni = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_muni.iterrows()])
paradas_bus_muni.crs = 'EPSG:3857'
paradas_bus_muni = paradas_bus_muni.to_crs('EPSG:4326')
paradas_bus_muni['MEDIO'] = 'COL'
paradas_bus_muni.head()

,LINEA,geometry,MEDIO
0,501,POINT (-58.38407 -34.88737),COL
1,501,POINT (-58.38771 -34.88767),COL
2,501,POINT (-58.39012 -34.88754),COL
3,501,POINT (-58.39024 -34.89055),COL
4,501,POINT (-58.39197 -34.89216),COL


In [47]:
paradas_bus_muni.LINEA.unique()

array([501, 505, 506, 510, 514, 515, 603, 619, 502, 504, 670, 741, 749,
       620, 621, 622, 628, 630, 520, 521, 526, 527, 540, 541, 542, 543,
       544, 548, 549, 550, 551, 552, 553, 561, 562, 564, 503, 634, 635,
       580, 582, 583, 584, 585, 710, 707, 720, 721, 723, 500, 624, 740,
       570, 511, 507, 523, 524])

In [48]:
paradas = pd.concat([paradas_bus_muni,paradas_bus_prov,paradas_bus_nac,ffcc,subte])
paradas.head()

,LINEA,geometry,MEDIO
0,501,POINT (-58.38407 -34.88737),COL
1,501,POINT (-58.38771 -34.88767),COL
2,501,POINT (-58.39012 -34.88754),COL
3,501,POINT (-58.39024 -34.89055),COL
4,501,POINT (-58.39197 -34.89216),COL


In [51]:
paradas.to_file('../carto/paradas.geojson',driver='GeoJSON')

In [52]:
paradas['LATITUDE'] = paradas.geometry.y
paradas['LONGITUDE'] = paradas.geometry.x
paradas.drop('geometry',axis=1,inplace=True)

In [53]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [54]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'
    .format(DB_USERNAME, DB_PASSWORD, DB_HOST,
            DB_PORT, DB_NAME))

In [55]:
%time h3_paradas = h3_indexing(paradas.copy(),res_list = [5,12])

CPU times: user 40 s, sys: 116 ms, total: 40.1 s
Wall time: 40.1 s


In [56]:
h3_paradas.head()

,LINEA,MEDIO,LATITUDE,LONGITUDE,h3_res_5,h3_res_6,h3_res_7,h3_res_8,h3_res_9,h3_res_10,h3_res_11,h3_res_12
0,501,COL,-34.887373,-58.384068,85c2e39bfffffff,86c2e3997ffffff,87c2e3994ffffff,88c2e39945fffff,89c2e399453ffff,8ac2e39945affff,8bc2e39945adfff,8cc2e39945ad1ff
1,501,COL,-34.887670,-58.387713,85c2e39bfffffff,86c2e3997ffffff,87c2e3994ffffff,88c2e39945fffff,89c2e3994cfffff,8ac2e3994cdffff,8bc2e3994cdbfff,8cc2e3994cdb1ff
2,501,COL,-34.887542,-58.390125,85c2e39bfffffff,86c2e3997ffffff,87c2e3994ffffff,88c2e39941fffff,89c2e39941bffff,8ac2e3994197fff,8bc2e39941b3fff,8cc2e3994194bff
3,501,COL,-34.890546,-58.390244,85c2e39bfffffff,86c2e3997ffffff,87c2e3994ffffff,88c2e3994dfffff,89c2e3994c7ffff,8ac2e3994c77fff,8bc2e3994c76fff,8cc2e3994c763ff
4,501,COL,-34.892161,-58.391970,85c2e39bfffffff,86c2e3997ffffff,87c2e3994ffffff,88c2e3994dfffff,89c2e3994d7ffff,8ac2e3994d47fff,8bc2e3994d45fff,8cc2e3994d45bff


In [57]:
h3_paradas.to_csv('../data/h3_paradas.csv',index=False)

In [58]:
h3_paradas.to_sql('paradas', engine, schema=DB_SCHEMA)

In [ ]:
# plotear el mapa de hexagrillas de paradas
lista_indices_global = h3_paradas.h3_res_5.unique()
geo_df = gpd.GeoDataFrame(lista_indices_global,
                          geometry = [Polygon(h3.h3_to_geo_boundary(h3_address=h, geo_json=True)
                                                                   ) for h in lista_indices_global],
                          crs = 'EPSG:4326')
geo_df.columns=['h3_index','geometry']
geo_df.to_file('../carto/carto_paradas_hex_res_5.geojson',driver='GeoJSON')
geo_df.head()